# Scenario Integration Test

Лаботоратория по искусственному интеллекту, Сбербанк. 

О чем: вызов сценариев с разными моделями.
В качестве датасета используется датасет MovieLens100K. 

## Содержание

1. [Импорты, создание спарк-сессии](#intro)
2. [Загрузка данных](#data-loader)
3. [Сценарии с разными моделями](#scenario)
3.1 [Получение сценария через фабрику](#get-scenario)
3.2 [Обучение сценария](#fit-scenario)

### Импорты, создание спарк-сессии <a name='intro'></a>

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import logging
import os
import sys
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
from sponge_bob_magic.datasets.movielens import MovieLens
from sponge_bob_magic.data_preparator.data_preparator import DataPreparator

from sponge_bob_magic.splitters import log_splitter
from sponge_bob_magic.splitters import user_log_splitter
from sponge_bob_magic import metrics

from sponge_bob_magic.models.popular_recomennder import PopularRecommender
from sponge_bob_magic.models.als_recommender import ALSRecommender
from sponge_bob_magic.models.knn_recommender import KNNRecommender
from sponge_bob_magic.models.lightfm_recommender import LightFMRecommender

from sponge_bob_magic.scenarios.main_scenario.main_factory import MainScenarioFactory
from sponge_bob_magic.utils import  get_spark_session
from sponge_bob_magic.constants import DEFAULT_CONTEXT
from pyspark.sql.functions import lit

In [3]:
# отображение максимальной ширины колонок в pandas датафреймах
pd.options.display.max_colwidth = -1

In [4]:
spark = get_spark_session()
spark

## Загрузка данных <a name="data-loader"></a>

In [5]:
data = MovieLens("100k")
log = spark.createDataFrame(data.ratings).withColumn(
    "context", lit(DEFAULT_CONTEXT)
)
data.info()

ratings


,user_id,item_id,relevance,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116



users


,user_id,gender,age,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067



items


,item_id,title,release_date,imdb_url,unknown,...,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1995-01-01,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,...,0,0,0,0,0
1,2,GoldenEye (1995),1995-01-01,http://us.imdb.com/M/title-exact?GoldenEye%20(1995),0,...,0,0,1,0,0
2,3,Four Rooms (1995),1995-01-01,http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995),0,...,0,0,1,0,0


## Сценарии с разными моделями <a name="scenario"></a>

### Получение сценария через фабрику <a name="get-scenario"></a>

In [6]:
popular_recommender = PopularRecommender()
als_recommender = ALSRecommender()
knn_recommender = KNNRecommender()
lightfm_recommender = LightFMRecommender()

In [7]:
log_bydate_splitter = log_splitter.LogSplitByDateSplitter(
    True, True,
    datetime(2007, 1, 1)
)
log_random_splitter = log_splitter.LogSplitRandomlySplitter(
    True, True,
    test_size=0.3, seed=1234
)
log_cold_splitter = log_splitter.ColdUsersExtractingSplitter(
    True, True,
    test_size=0.3
)
user_random_splitter = user_log_splitter.RandomUserLogSplitter(
    True, True,
    item_test_size=0.3,
    seed=1234,
    user_test_size=500
)
user_bydate_splitter = user_log_splitter.ByTimeUserLogSplitter(
    True, True,
    item_test_size=0.3,
    seed=1234,
    user_test_size=500
)

### Обучение сценария <a name="fit-scenario"></a>

In [13]:
results = None

In [9]:
scenario = MainScenarioFactory().get(
    splitter=user_random_splitter,
    recommender=lightfm_recommender,
    criterion=metrics.HitRate(),
    metrics=[
        metrics.NDCG(),
        metrics.Precision(),
        metrics.MAP(),
        metrics.Recall(),
        metrics.Surprisal(log),
    ],
    fallback_recommender=popular_recommender,
)

In [10]:
popular_grid = {
    "alpha": {"type": "int", "args": [0, 10]},
    "beta": {"type": "int", "args": [0, 10]}
}
als_grid = {
    "rank": {"type": "discrete_uniform", "args": [10, 100, 10]}
}
lightfm_grid = {
    "rank": {"type": "int", "args": [10, 100]}
}
knn_grid = {
    "shrink": {"type": "discrete_uniform", "args": [10, 50, 10]},
    "num_neighbours": {"type": "discrete_uniform", "args": [0, 10, 1]},
}

In [11]:
best_params = scenario.research(
    lightfm_grid,
    log,
    k=10,
    n_trials=2
)

05-Feb-20 17:29:58, root, DEBUG: Деление лога на обучающую и тестовую выборку
05-Feb-20 17:30:18, root, DEBUG: Длина трейна и теста: (84013, 15944)
05-Feb-20 17:30:19, root, DEBUG: Количество пользователей в трейне и тесте: 943, 490
05-Feb-20 17:30:20, root, DEBUG: Количество объектов в трейне и тесте: 1644, 1355
05-Feb-20 17:30:20, root, DEBUG: Обучение и предсказание дополнительной модели
05-Feb-20 17:30:20, root, DEBUG: Проверка датафреймов
05-Feb-20 17:30:21, root, DEBUG: Предварительная стадия обучения (pre-fit)
05-Feb-20 17:30:22, root, DEBUG: Среднее количество items у каждого user: 90
05-Feb-20 17:30:25, root, DEBUG: Основная стадия обучения (fit)
05-Feb-20 17:30:25, root, DEBUG: Проверка датафреймов
05-Feb-20 17:30:27, root, DEBUG: Количество items после фильтрации: 100
05-Feb-20 17:30:31, root, DEBUG: Пре-фит модели
05-Feb-20 17:30:32, root, DEBUG: -------------
05-Feb-20 17:30:32, root, DEBUG: Оптимизация параметров
05-Feb-20 17:30:32, root, DEBUG: Максимальное количество по

In [14]:
results = pd.concat([scenario.study.trials_dataframe(), results], axis=0)

results

,number,value,datetime_start,datetime_complete,params_rank,user_attrs_MAP@k,user_attrs_Precision@k,user_attrs_Recall@K,user_attrs_Surprisal@K,user_attrs_nDCG@k,system_attrs__number,state
0,0,0.865306,2020-02-05 17:30:32.205792,2020-02-05 17:31:24.068727,12,0.045775,0.235714,0.102034,1.467280,0.253048,0,COMPLETE
1,1,0.840816,2020-02-05 17:31:24.164692,2020-02-05 17:32:15.171145,83,0.044364,0.230612,0.098064,1.722532,0.243003,1,COMPLETE


### Получение рекомендаций <a name="predict-scenario"></a>

In [15]:
recs = scenario.production(
    best_params, 
    log,
    users=None, 
    items=None,
    k=10
)

05-Feb-20 17:32:32, root, DEBUG: Проверка датафреймов
05-Feb-20 17:32:33, root, DEBUG: Предварительная стадия обучения (pre-fit)
05-Feb-20 17:32:33, root, DEBUG: Основная стадия обучения (fit)
05-Feb-20 17:32:33, root, DEBUG: Построение модели LightFM
05-Feb-20 17:32:34, root, DEBUG: Проверка датафреймов
05-Feb-20 17:32:35, root, DEBUG: Выделение дефолтных юзеров
05-Feb-20 17:32:35, root, DEBUG: Выделение дефолтных айтемов


In [16]:
recs.show()

+-------+-------+--------------------+----------+
|user_id|item_id|           relevance|   context|
+-------+-------+--------------------+----------+
|     91|    199| 0.23271000385284424|no_context|
|     91|    483| 0.04666692018508911|no_context|
|     91|    194|-0.04420399665832...|no_context|
|     91|    435| -0.2268085479736328|no_context|
|     91|    197|-0.28308582305908203|no_context|
|     91|    480|-0.31120729446411133|no_context|
|     91|    661|-0.36454081535339355|no_context|
|     91|    511| -0.3815042972564697|no_context|
|     91|    510|  -0.413942813873291|no_context|
|     91|    205| -0.4170534014701843|no_context|
|    152|    393| -0.7733075022697449|no_context|
|    152|     94| -0.9557117223739624|no_context|
|    152|     88| -1.0421855449676514|no_context|
|    152|    419| -1.0703446865081787|no_context|
|    152|    739|  -1.152461051940918|no_context|
|    152|    402|  -1.180585503578186|no_context|
|    152|     66|  -1.207716703414917|no_context|
